In [1]:
# Test system:
# Mac + Python 2 + conda activate superfold_x86

## Section 1a. Simple RNA Fold for given sequence (no SHAPE constraint)

In [2]:
import subprocess
import os

def predict_rna_structure(rna_sequence, rna_name, output_dir='output_structure'):
    '''
    Predicts secondary structure of RNA sequence using RNAstructure package.

    Parameters:
    - rna_sequence (str): The RNA sequence to analyze.
    - rna_name (str): The name identifier for the RNA sequence (used for file naming).
    - output_dir (str, optional): The directory where output files will be stored. Defaults to 'output_structure'.

    Returns:
    - str: The RNA secondary structure in dot-bracket notation.
    - Files: *.seq (fasta format), *.ct (energy + connectivity), *.dbn (energy + seq + dot bracket notion).

    Note:
    - change environment variables for RNAstructure if necessary.
    '''
    # Set the environment variables for RNAstructure
    os.environ['DATAPATH'] = os.path.expanduser('~/RNAstructure/data_tables')
    os.environ['PATH'] = os.path.expanduser('~/RNAstructure/exe') + os.pathsep + os.environ['PATH']
    
    # Ensure RNA sequence is in uppercase and RNA format (U instead of T)
    rna_sequence = rna_sequence.upper().replace('T', 'U')

    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define input and output file paths using rna_name
    input_file = os.path.join(output_dir, '{}.seq'.format(rna_name))
    ct_file = os.path.join(output_dir, '{}.ct'.format(rna_name))
    dot_bracket_file = os.path.join(output_dir, '{}.dbn'.format(rna_name))

    # Write RNA sequence to input file in FASTA-like format
    with open(input_file, 'w') as f:
        f.write(">{}\n{}\n".format(rna_name, rna_sequence))

    # Run RNAstructure Fold command to predict structure
    fold_result = subprocess.call(['Fold', input_file, ct_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if fold_result != 0:
        print("Error running Fold. Check if RNAstructure is properly installed.")
        return None

    # Convert the CT file to dot-bracket notation to get base-pairing info
    ct2dot_result = subprocess.call(['ct2dot', ct_file, '1', dot_bracket_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if ct2dot_result != 0:
        print("Error running ct2dot. Check if RNAstructure is properly installed.")
        return None

    # Read and return the dot-bracket notation (base-pairing info)
    if os.path.exists(dot_bracket_file):
        with open(dot_bracket_file, 'r') as f:
            dot_bracket_content = f.read()
        return dot_bracket_content
    else:
        print("Dot-bracket file was not created.")
        return None

In [3]:
rna_seq = "GAAGGTGCTCACGACTTATTCCTTGCTAGCTAGTGGAAGGAAGGCACTGTGTCGTTTACGACACGACTGAAGGAAGGAGTCAGCTAGCAAGTGGTAAGTCTGCCAGCATTAT"
rna_name = "TG"
structure_info = predict_rna_structure(rna_seq, rna_name, "test")
if structure_info:
    print(structure_info)

>ENERGY = -43.4  TG
GAAGGUGCUCACGACUUAUUCCUUGCUAGCUAGUGGAAGGAAGGCACUGUGUCGUUUACGACACGACUGAAGGAAGGAGUCAGCUAGCAAGUGGUAAGUCUGCCAGCAUUAU
...((((((((.((((((((.((((((((((.(((.........))).((((((....))))))((((.........)))))))))))))).))))))))))..))))))..



## Section 1b. RNA Fold for given sequence and yield probability info  (no SHAPE constraint)

In [4]:
import subprocess
import os

def predict_rna_structure_prob(rna_sequence, rna_name, output_dir='output_structure'):
    '''
    Predicts secondary structure of RNA sequence using RNAstructure package.

    Parameters:
    - rna_sequence (str): The RNA sequence to analyze.
    - rna_name (str): The name identifier for the RNA sequence (used for file naming).
    - output_dir (str, optional): The directory where output files will be stored. Defaults to 'output_structure'.

    Returns:
    - str: The RNA secondary structure in dot-bracket notation.
    - files: *.seq (fasta format), *.ct (energy + connectivity), *.pfs (base-pairing probability matrix),
            *.prob/*.dp (probability).

    Note:
    - change environment variables for RNAstructure if necessary.
    '''
    # Set the environment variables for RNAstructure
    os.environ['DATAPATH'] = os.path.expanduser('~/RNAstructure/data_tables')
    os.environ['PATH'] = os.path.expanduser('~/RNAstructure/exe') + os.pathsep + os.environ['PATH']
        
    # Ensure RNA sequence is in uppercase and RNA format (U instead of T)
    rna_sequence = rna_sequence.upper().replace('T', 'U')

    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define input and output file paths using rna_name
    input_file = os.path.join(output_dir, '{}.seq'.format(rna_name))
    ct_file = os.path.join(output_dir, '{}.ct'.format(rna_name))
    prob_file = os.path.join(output_dir, '{}.pfs'.format(rna_name))
    dp_file = os.path.join(output_dir, '{}.dp'.format(rna_name))
    prob_output_file = os.path.join(output_dir, '{}.prob'.format(rna_name))

    # Write RNA sequence to input file in FASTA-like format
    with open(input_file, 'w') as f:
        f.write(">{}\n{}\n".format(rna_name, rna_sequence))

    # Run RNAstructure Fold command to predict structure
    fold_result = subprocess.call(['Fold', input_file, ct_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if fold_result != 0:
        print("Error running Fold. Check if RNAstructure is properly installed.")
        return None

    # Run Partition to calculate base-pairing probabilities
    partition_result = subprocess.call(['Partition', input_file, prob_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if partition_result != 0:
        print("Error running Partition. Check if RNAstructure is properly installed.")
        return None

    # Generate the .prob file using ProbabilityPlot with the --text option
    generate_prob_file(prob_file, prob_output_file)

    # Optionally convert the .prob file to .dp format
    convert_prob_to_dp(prob_output_file, dp_file)

    print("Dot plot file generated: {}".format(dp_file))

def generate_prob_file(pfs_file, prob_file):
    """
    Convert the .pfs file to .prob format (text-based probability plot) using RNAstructure's ProbabilityPlot command with --text option.
    """
    # Call RNAstructure ProbabilityPlot tool with the --text option to generate the .prob file
    prob_result = subprocess.call(['ProbabilityPlot', '--text', pfs_file, prob_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if prob_result != 0:
        print("Error running ProbabilityPlot with --text. Check if RNAstructure is properly installed.")
        return None

    print("Text-based probability plot saved to: {}".format(prob_file))

def convert_prob_to_dp(prob_file, dp_file):
    """
    Convert the .prob file to .dp format manually by copying the contents.
    """
    # Since .dp and .prob formats are similar, we can directly copy the data
    with open(prob_file, 'r') as prob, open(dp_file, 'w') as dp:
        for line in prob:
            dp.write(line)
    
    print("Converted .prob to .dp format and saved to: {}".format(dp_file))

In [5]:
# Example usage
rna_sequence = "GGTTTTAGACAAAATCAAAAAGAAGGAAGGTGCTCACGACTTATTCCTTGCTAGCTAGTGGAAGGAAGGCACTGTGTCGTTTACGACACGACTGAAGGAAGGAGTCAGCTAGCAAGAAGTAAGTCTGCCAGCATTATGAAA"
rna_name = "AA"
predict_rna_structure_prob(rna_sequence, rna_name, "test")

Text-based probability plot saved to: test/AA.prob
Converted .prob to .dp format and saved to: test/AA.dp
Dot plot file generated: test/AA.dp


## Section 2a. Generate arc plot (PDF)

In [6]:
# PDF version not working well for long RNAs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.patches import Arc  # Import Arc from matplotlib.patches

def generate_arc_plot(start_nt, end_nt, dp_file, name):
    '''
    Generates an arc diagram visualization of RNA base-pairing interactions based on a dot plot file (.dp).

    Parameters:
    - start_nt (int): The starting nucleotide position for the plot.
    - end_nt (int): The ending nucleotide position for the plot.
    - dp_file (str): Path to the dot plot (.dp) file containing base-pairing probabilities.
    - name (str): Name used for saving the output PDF file.

    Categorizes arcs into four probability-based groups, using different colors:
        - Green: Highest probability (`log_prob <= 0.09691`)
        - Blue: Medium-high probability (`0.09691 < log_prob <= 0.52288`)
        - Yellow: Medium probability (`0.52288 < log_prob <= 1`)
        - Light Grey: Low probability (`log_prob > 1`)

    Output:
    - Saves a `.pdf` file visualizing base-pairing probabilities in an arc diagram.
    '''
    # Step 1: Read the dp file
    with open(dp_file, 'r') as f:
        # Read total length from the first line
        total_length = int(f.readline().strip())
        
        # Read the probability data into a DataFrame
        prob_df = pd.read_csv(f, sep='\t')
    
    # Remove rows with log_prob >= 2
    prob_df = prob_df[prob_df['-log10(Probability)'] < 2]
    
    # Step 2: Check if end_nt <= total_length
    if end_nt > total_length:
        raise ValueError("end_nt exceeds the total length of the RNA sequence in the dp file")
    
    # Step 3: Calculate figure size based on nucleotide range and maximum arc height
    # Filter the dataframe to include only rows where i and j are within the given range
    filtered_prob_df = prob_df[(prob_df['j'] <= end_nt) & (prob_df['i'] >= start_nt)]
    # Compute max_arc_height based on the filtered DataFrame
    if not filtered_prob_df.empty:
        max_arc_height = max(filtered_prob_df['j'] - filtered_prob_df['i']) / 2.0
    else:
        max_arc_height = 0  # Default to 0 if no valid arcs exist in the range
    fig_width = (end_nt - start_nt) / 4
    fig_height = (max_arc_height) / 4
    
    # Create the x-axis: nucleotide range from start_nt to end_nt
    x_range = range(start_nt, end_nt + 1)
    
    # Set up the plot with dynamic figure size
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    ax.set_xlim(start_nt - 0.5, end_nt + 0.5)
    ax.set_ylim(0, max_arc_height + 0.5)  # Use the maximum arc height for y-axis limit
    ax.set_xticks(x_range)
    
    # Collect all arcs with their priorities
    arc_list = []
    
    # Step 4: Check each row in prob_df, and prepare arcs with valid (i, j) within the start_nt and end_nt range
    for _, row in prob_df.iterrows():
        i, j, log_prob = int(row['i']), int(row['j']), row['-log10(Probability)']
        
        # Check if both i and j are within the specified nucleotide range
        if start_nt <= i <= end_nt and start_nt <= j <= end_nt:
            # Calculate the radius and center of the arc (use float division for center)
            radius = (j - i) / 2.0  # Use float division
            center = (i + j) / 2.0  # Use float division for the midpoint
            
            # Determine color based on the -log10(Probability)
            if log_prob <= 0.09691:
                color = 'green'
                priority = 3  # Highest priority (draw last)
            elif log_prob <= 0.52288:
                color = '#1C73BE'  # Greyish blue
                priority = 2  # Blue has second highest priority
            elif log_prob <= 1:
                color = '#F7CE12'  # Yellow
                priority = 1  # Yellow is drawn after grey
            else:
                color = '#D3D3D3'  # Very light grey
                priority = 0  # Lowest priority (draw first)
            
            # Append the arc details with priority
            arc_list.append((Arc((center, 0), j - i, j - i, theta1=0, theta2=180, color=color, lw=10, alpha=0.5), priority))
    
    # Step 5: Sort the arcs by priority (lower priority drawn first)
    arc_list.sort(key=lambda x: x[1])  # Sort based on priority
    
    # Step 6: Add arcs to the plot in sorted order
    for arc, _ in arc_list:
        ax.add_patch(arc)
    
    # Step 7: Customize plot appearance
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.yaxis.set_ticks([])  # Remove y-axis ticks
    ax.tick_params(axis='x', labelsize=4)  # Set tick label size to 8 (adjust as needed)
    ax.set_aspect('equal')
    
    # Step 8: Save the plot as a PDF with the name '{name}.dp.pdf' using .format()
    # Pass dpi=300 for better resolution and ensure the figure size is preserved
    plt.savefig('{}.pdf'.format(name), format='pdf', bbox_inches='tight', dpi=300)
    #plt.show()
    plt.close(fig) # not showing on this page to save space

In [7]:
# Example usage
generate_arc_plot(1, 20, 'test/AA.dp', 'test/AA')

## Section 2a. Generate arc plot (SVG)

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.patches import Arc  # Import Arc from matplotlib.patches

def generate_arc_plot(start_nt, end_nt, dp_file, name):
    '''
    Generates an arc diagram visualization of RNA base-pairing interactions based on a dot plot file (.dp).

    Parameters:
    - start_nt (int): The starting nucleotide position for the plot.
    - end_nt (int): The ending nucleotide position for the plot.
    - dp_file (str): Path to the dot plot (.dp) file containing base-pairing probabilities.
    - name (str): Name used for saving the output PDF file.

    Categorizes arcs into four probability-based groups, using different colors:
        - Green: Highest probability (`log_prob <= 0.09691`)
        - Blue: Medium-high probability (`0.09691 < log_prob <= 0.52288`)
        - Yellow: Medium probability (`0.52288 < log_prob <= 1`)
        - Light Grey: Low probability (`log_prob > 1`)

    Output:
    - Saves a `.svg` file visualizing base-pairing probabilities in an arc diagram.
    '''
    # Step 1: Read the dp file
    with open(dp_file, 'r') as f:
        # Read total length from the first line
        total_length = int(f.readline().strip())
        
        # Read the probability data into a DataFrame
        prob_df = pd.read_csv(f, sep='\t')
    
    # Remove rows with log_prob >= 2
    prob_df = prob_df[prob_df['-log10(Probability)'] < 2]
    
    # Step 2: Check if end_nt <= total_length
    if end_nt > total_length:
        raise ValueError("end_nt exceeds the total length of the RNA sequence in the dp file")
    
    # Step 3: Calculate figure size based on nucleotide range and maximum arc height
    filtered_prob_df = prob_df[(prob_df['j'] <= end_nt) & (prob_df['i'] >= start_nt)]
    # Compute max_arc_height based on the filtered DataFrame
    if not filtered_prob_df.empty:
        max_arc_height = max(filtered_prob_df['j'] - filtered_prob_df['i']) / 2.0
    else:
        max_arc_height = 0  # Default to 0 if no valid arcs exist in the range
    fig_width = (end_nt - start_nt) / 4
    fig_height = max_arc_height / 4
    
    # Create the x-axis: nucleotide range from start_nt to end_nt
    x_range = range(start_nt, end_nt + 1)
    
    # Set up the plot with dynamic figure size
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    ax.set_xlim(start_nt - 0.5, end_nt + 0.5)
    ax.set_ylim(0, max_arc_height + 0.5)  # Use the maximum arc height for y-axis limit
    ax.set_xticks(x_range)
    
    # Collect all arcs with their priorities
    arc_list = []
    
    # Step 4: Check each row in prob_df, and prepare arcs with valid (i, j) within the start_nt and end_nt range
    for _, row in prob_df.iterrows():
        i, j, log_prob = int(row['i']), int(row['j']), row['-log10(Probability)']
        
        # Check if both i and j are within the specified nucleotide range
        if start_nt <= i <= end_nt and start_nt <= j <= end_nt:
            # Calculate the radius and center of the arc (use float division for center)
            radius = (j - i) / 2.0  # Use float division
            center = (i + j) / 2.0  # Use float division for the midpoint
            
            # Determine color based on the -log10(Probability)
            if log_prob <= 0.09691:
                color = 'green'
                priority = 3  # Highest priority (draw last)
            elif log_prob <= 0.52288:
                color = '#1C73BE'  # Greyish blue
                priority = 2  # Blue has second highest priority
            elif log_prob <= 1:
                color = '#F7CE12'  # Yellow
                priority = 1  # Yellow is drawn after grey
            else:
                color = '#D3D3D3'  # Very light grey
                priority = 0  # Lowest priority (draw first)
            
            # Append the arc details with priority
            arc_list.append((Arc((center, 0), j - i, j - i, theta1=0, theta2=180, color=color, lw=10, alpha=0.5), priority))
    
    # Step 5: Sort the arcs by priority (lower priority drawn first)
    arc_list.sort(key=lambda x: x[1])  # Sort based on priority
    
    # Step 6: Add arcs to the plot in sorted order
    for arc, _ in arc_list:
        ax.add_patch(arc)
    
    # Step 7: Customize plot appearance
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.yaxis.set_ticks([])  # Remove y-axis ticks
    ax.tick_params(axis='x', labelsize=4) 
    ax.set_aspect('equal')

    # Step 8: Save the plot as an SVG with the name '{name}.svg' using .format()
    plt.savefig('{}.svg'.format(name), format='svg', bbox_inches='tight')
    #plt.show()
    plt.close(fig) # not showing on this page to save space 

In [9]:
# Example usage
generate_arc_plot(1, 20, 'test/AA.dp', 'test/AA')